In [1]:
import redis

# Connect to Redis (adjust host and port if necessary)
r = redis.Redis(host="redis", port=6379, db=3) 


In [2]:
print(r)

<redis.client.Redis(<redis.connection.ConnectionPool(<redis.connection.Connection(host=redis,port=6379,db=3)>)>)>


In [3]:
r.keys()

[b'005-000107715',
 b'005-000136735',
 b'001-000263524',
 b'005-000088989',
 b'005-000157919',
 b'005-000126722',
 b'002-000217889',
 b'001-000248269',
 b'001-000255285',
 b'005-000136553',
 b'004-000183530',
 b'005-000109305',
 b'003-000209184',
 b'003-000186881',
 b'003-000177427',
 b'005-000147541',
 b'002-000105004',
 b'005-000104373',
 b'001-000144249',
 b'004-000111979',
 b'001-000163141',
 b'004-000137286',
 b'003-000131827',
 b'004-000177033',
 b'002-000144107',
 b'002-000115912',
 b'003-000099402',
 b'001-000181897',
 b'001-000169216',
 b'002-000115446',
 b'002-000203726',
 b'005-000105691',
 b'003-000154102',
 b'002-000109083',
 b'002-000158290',
 b'004-000138546',
 b'004-000115633',
 b'004-000177233',
 b'003-000183488',
 b'002-000119996',
 b'001-000179181',
 b'002-000218323',
 b'003-000134868',
 b'003-000131547',
 b'004-000089887',
 b'001-000250031',
 b'order:12345',
 b'004-000143587',
 b'003-000164376',
 b'001-000128112',
 b'004-000192337']

In [4]:
import time
import random
import redis
import json



# Sample JSON data (shortened for example)
customers = [
    {
        "customer_id": 563,
        "first_name": "Rose",
        "last_name": "Slimings",
        "street": "38 Iowa Street",
        "city": "Berkeley",
        "state": "CA",
        "zip": "94704",
        "closest_store_id": 1,
        "distance": 1,
    }
]

stores = [
    {
        "store_id": 1,
        "street": "3000 Telegraph Ave",
        "city": "Berkeley",
        "state": "CA",
        "zip": "94705",
        "latitude": 37.8555,
        "longitude": -122.2604,
    }
]

# Simulated function to publish delivery updates
def publish_delivery_update(order_id, status, location, coordinates):
    # Define message payload
    message = {
        "order_id": order_id,
        "status": status,
        "location": location,
        "coordinates": coordinates,
        "timestamp": time.time(),
    }
    # Convert message dictionary to JSON string
    message_json = json.dumps(message)
    # Publish the message to a Redis channel
    r.publish("delivery_updates", message_json)
    #incorporate robot/drone to receive published channel data for real-time updates
    # Save the update in a hash for persistence, converting to JSON string
    r.hset(f"order:{order_id}", mapping={k: json.dumps(v) for k, v in message.items()})

# Get a sample customer and store data
customer = customers[0]
store = stores[0]

# Simulate delivery updates with relevant locations and coordinates
order_id = "12345"
statuses = ["Dispatched", "In Transit", "Arriving", "Delivered"]
locations = [
    "Warehouse", 
    f"En route from {store['street']}, {store['city']}", 
    f"Near {customer['street']}, {customer['city']}", 
    f"Customer's Door at {customer['street']}, {customer['city']}"
]
coordinates = [
    {"lat": store["latitude"], "lon": store["longitude"]},  # Warehouse
    {"lat": (store["latitude"] + customer["distance"] * 0.001), "lon": (store["longitude"] + customer["distance"] * 0.001)},  # In Transit
    {"lat": store["latitude"], "lon": store["longitude"]},  # Near customer
    {"lat": store["latitude"], "lon": store["longitude"]}  # Customer's door
]
# Define simulated time delays (in seconds) for each status
time_delays = {
    "Dispatched": random.randint(5, 10), 
    "In Transit": random.randint(10, 15),  
    "Arriving": random.randint(3, 6),  
    "Delivered": random.randint(1, 3)  
}

# Publishing simulated delivery updates with dynamic delays
for status, location, coordinate in zip(statuses, locations, coordinates):
    publish_delivery_update(order_id, status, location, coordinate)
    print(
        f"Update published: Order {order_id} - Status: {status} - Location: {location} - Coordinates: {coordinate}"
    )
    delay = time_delays[status]
    print(f"Simulating delay for {status}: {delay} minutes")
    time.sleep(delay)  # Apply the specific delay for each status update



Update published: Order 12345 - Status: Dispatched - Location: Warehouse - Coordinates: {'lat': 37.8555, 'lon': -122.2604}
Simulating delay for Dispatched: 8 minutes
Update published: Order 12345 - Status: In Transit - Location: En route from 3000 Telegraph Ave, Berkeley - Coordinates: {'lat': 37.8565, 'lon': -122.2594}
Simulating delay for In Transit: 14 minutes
Update published: Order 12345 - Status: Arriving - Location: Near 38 Iowa Street, Berkeley - Coordinates: {'lat': 37.8555, 'lon': -122.2604}
Simulating delay for Arriving: 3 minutes
Update published: Order 12345 - Status: Delivered - Location: Customer's Door at 38 Iowa Street, Berkeley - Coordinates: {'lat': 37.8555, 'lon': -122.2604}
Simulating delay for Delivered: 2 minutes
